In [28]:
from requests_html import HTMLSession
import pandas as pd
from random import choice
from time import sleep
import itertools    #جهت تبدیل لیست چند بعدی تولید شده در تابع دریافت کننده لیست صفحات بعد

In [29]:
def next(count=8):
    '''دریافت کد صفحات حاوی اطلاعات مورد نیاز'''
    count = count
    next_pages=[0]
    session = HTMLSession()
    while count!= 0:
        response=session.get('https://api.divar.ir/v8/map-discovery/bbox/posts?lon1=51.09336767388771&lat1=35.570926635221014&lon2=51.64411325946142&lat2=35.872539895122685&nextPageId=0&filters=building-age=0-10&filters=category=apartment-sell&filters=size=60-110')
        if response.json().get("next_page_id") != None:
            next_pages.append(response.json().get("next_page_id"))
        count-=1
    return next_pages

In [30]:
apartment = next(8)
urls=[f'https://api.divar.ir/v8/map-discovery/bbox/posts?lon1=51.09336767388771&lat1=35.570926635221014&lon2=51.64411325946142&lat2=35.872539895122685&nextPageId={_}&filters=building-age=0-10&filters=category=apartment-sell&filters=size=65-110' for _ in apartment]
def tokens(url):
    '''دریافت توکن های موجود در صفحات ای پی آی دیوار'''
    session = HTMLSession()
    response=session.get(url)
    datas=response.json()["posts"]
    # for _ in datas:
    data=[_.get("token") for _ in datas]
    return data   #ساخت لیست از همه توکن ها به صورت چند بعدی
tokenz = list(itertools.chain(*list(map(tokens,urls)))) #مسطح کردن لیست توکن

In [ ]:
urls=[f"https://divar.ir/v/{token}" for token in tokenz]
# Create an empty dataframe with the desired columns
df = pd.DataFrame(columns=["location", "price_per_meter"])
def main(url):
    session=HTMLSession()
    user_agent=choice([
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36",
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/115.0",
            "Mozilla/5.0 (Macintosh; Intel Mac OS X 13.4; rv:109.0) Gecko/20100101 Firefox/115.0",
            "Mozilla/5.0 (X11; Linux i686; rv:109.0) Gecko/20100101 Firefox/115.0",
            "Mozilla/5.0 (iPhone; CPU iPhone OS 13_4_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) FxiOS/115.0 Mobile/15E148 Safari/605.1.15",
            "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.360",
            "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_4_1) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.5 Safari/605.1.15",
            'Mozilla/5.0 (Windows NT 10.0; Trident/7.0; rv:11.0) like Gecko',
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36 Edg/114.0.1823.86',
            "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_4_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36 OPR/100.0.0.0",
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36 Vivaldi/6.1.3035.111",

        ])
    headers={'User-Agent':user_agent}
    response=session.get(url,headers=headers)
    if response.status_code ==429:
        sleep(2)
        response=session.get(url,headers=headers)
    title=response.html.xpath("//*[@id='app']/div[2]/div/div[1]/div[1]/div[1]/div/div[2]/text()")
    price_per_meter=response.html.xpath('//*[@id="app"]/div[2]/div/div[1]/div[1]/div[4]/div[3]/div[2]/p/text()')
    if title!=[]:
        location=str(title).split("در")[1]
    else:
        location=title
    # Return a dictionary with the location and price_per_meter as keys and values
    return {"location": location, "price_per_meter": price_per_meter}
# Use list comprehension to apply the main function to each url and get a list of dictionaries
# results = [main(url) for url in urls]
results=list(map(main,urls))
# Append the results to the dataframe
df = pd.concat([df, pd.DataFrame(results)], ignore_index=True)
# Save the dataframe to excel file
df.to_excel("tehran_price.xlsx")